In [1]:
import pandas as pd
books = pd.read_csv('bookstore.csv')

print(books.head())

          Transaction
0    History,Bookmark
1    History,Bookmark
2    Fiction,Bookmark
3  Biography,Bookmark
4    History,Bookmark


In [2]:
transactions = books['Transaction'].apply(lambda t: t.split(','))

In [3]:
transactions = list(transactions)

In [4]:
transactions

[['History', 'Bookmark'],
 ['History', 'Bookmark'],
 ['Fiction', 'Bookmark'],
 ['Biography', 'Bookmark'],
 ['History', 'Bookmark'],
 ['Poetry', 'Bookmark'],
 ['Biography', 'Bookmark'],
 ['Poetry', 'Bookmark'],
 ['Biography', 'Bookmark'],
 ['Biography', 'Bookmark'],
 ['History', 'Bookmark'],
 ['Fiction', 'Bookmark'],
 ['History', 'Bookmark'],
 ['Biography', 'Bookmark'],
 ['Poetry', 'Bookmark'],
 ['Biography', 'Bookmark'],
 ['Fiction', 'Bookmark'],
 ['Biography', 'Bookmark'],
 ['Poetry', 'Bookmark'],
 ['History', 'Bookmark'],
 ['History', 'Bookmark'],
 ['Poetry', 'Bookmark'],
 ['Fiction', 'Bookmark'],
 ['History', 'Bookmark'],
 ['Biography', 'Bookmark'],
 ['Biography', 'Bookmark'],
 ['Fiction', 'Bookmark'],
 ['Biography', 'Bookmark'],
 ['History', 'Bookmark'],
 ['Fiction', 'Bookmark'],
 ['History', 'Bookmark'],
 ['History', 'Bookmark'],
 ['Biography', 'Bookmark'],
 ['Biography', 'Bookmark'],
 ['History', 'Bookmark'],
 ['History', 'Bookmark'],
 ['Biography', 'Bookmark'],
 ['Biography', 'B

In [5]:
from itertools import permutations

flattened = [item for transaction in transactions for item in transaction]
items = list(set(flattened))

In [6]:
items

['History', 'Biography', 'Fiction', 'Bookmark', 'Poetry']

In [7]:
rules = list(permutations(items, 2))

In [8]:
rules

[('History', 'Biography'),
 ('History', 'Fiction'),
 ('History', 'Bookmark'),
 ('History', 'Poetry'),
 ('Biography', 'History'),
 ('Biography', 'Fiction'),
 ('Biography', 'Bookmark'),
 ('Biography', 'Poetry'),
 ('Fiction', 'History'),
 ('Fiction', 'Biography'),
 ('Fiction', 'Bookmark'),
 ('Fiction', 'Poetry'),
 ('Bookmark', 'History'),
 ('Bookmark', 'Biography'),
 ('Bookmark', 'Fiction'),
 ('Bookmark', 'Poetry'),
 ('Poetry', 'History'),
 ('Poetry', 'Biography'),
 ('Poetry', 'Fiction'),
 ('Poetry', 'Bookmark')]

In [9]:
from mlxtend.preprocessing import TransactionEncoder

In [10]:
encoder = TransactionEncoder().fit(transactions)

In [11]:
onehot = encoder.transform(transactions)

In [12]:
onehot = pd.DataFrame(onehot, columns=encoder.columns_)

In [13]:
onehot

,Biography,Bookmark,Fiction,History,Poetry
0,False,True,False,True,False
1,False,True,False,True,False
2,False,True,True,False,False
3,True,True,False,False,False
4,False,True,False,True,False
...,...,...,...,...,...
94,True,True,False,False,False
95,False,True,False,False,True
96,False,True,False,True,False
97,True,True,False,False,False


In [14]:
# Support values
print(onehot.mean())

Biography    0.404040
Bookmark     1.000000
Fiction      0.252525
History      0.252525
Poetry       0.090909
dtype: float64


In [15]:
import numpy as np

onehot['fiction+Bookmark'] = np.logical_and(onehot['Fiction'], onehot['Bookmark'])

print(onehot.mean())

Biography           0.404040
Bookmark            1.000000
Fiction             0.252525
History             0.252525
Poetry              0.090909
fiction+Bookmark    0.252525
dtype: float64


In [19]:
onehot.drop(['fiction+Bookmark'], axis=1, inplace=True)

In [20]:
onehot

,Biography,Bookmark,Fiction,History,Poetry
0,False,True,False,True,False
1,False,True,False,True,False
2,False,True,True,False,False
3,True,True,False,False,False
4,False,True,False,True,False
...,...,...,...,...,...
94,True,True,False,False,False
95,False,True,False,False,True
96,False,True,False,True,False
97,True,True,False,False,False


In [30]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(onehot, min_support = 0.006, max_len = 3, use_colnames = True)

print(frequent_itemsets)

    support               itemsets
0  0.404040            (Biography)
1  1.000000             (Bookmark)
2  0.252525              (Fiction)
3  0.252525              (History)
4  0.090909               (Poetry)
5  0.404040  (Bookmark, Biography)
6  0.252525    (Bookmark, Fiction)
7  0.252525    (History, Bookmark)
8  0.090909     (Bookmark, Poetry)


In [33]:
rules = association_rules(frequent_itemsets, metric='support', min_threshold=0.0)

In [34]:
print(rules)

   antecedents  consequents  antecedent support  consequent support   support  \
0   (Bookmark)  (Biography)            1.000000            0.404040  0.404040   
1  (Biography)   (Bookmark)            0.404040            1.000000  0.404040   
2   (Bookmark)    (Fiction)            1.000000            0.252525  0.252525   
3    (Fiction)   (Bookmark)            0.252525            1.000000  0.252525   
4    (History)   (Bookmark)            0.252525            1.000000  0.252525   
5   (Bookmark)    (History)            1.000000            0.252525  0.252525   
6   (Bookmark)     (Poetry)            1.000000            0.090909  0.090909   
7     (Poetry)   (Bookmark)            0.090909            1.000000  0.090909   

   confidence  lift  leverage  conviction  
0    0.404040   1.0       0.0         1.0  
1    1.000000   1.0       0.0         inf  
2    0.252525   1.0       0.0         1.0  
3    1.000000   1.0       0.0         inf  
4    1.000000   1.0       0.0         inf  
5    0